In [13]:
import numpy as np
import numba
from numba import cuda, njit, vectorize
import time
import math
import torch

In [14]:
cuda.is_available()

True

In [3]:
def function(x):
    return math.sin(x) + 0.5 * math.sin(2*x) + 0.25

# Нативный питончик

In [4]:
def monte_carlo_python(n_points, x_min=0, x_max=2*math.pi, y_min=0, y_max=2):
    count_inside = 0
    
    for _ in range(n_points):
        x = np.random.uniform(x_min, x_max)
        y = np.random.uniform(y_min, y_max)
        
        if y <= function(x):
            count_inside += 1
    area_rectangle = (x_max - x_min) * (y_max - y_min)
    area_under_curve = area_rectangle * count_inside / n_points
    return area_under_curve

# Numba CPU

In [5]:
@njit
def function_numba(x):
    return math.sin(x) + 0.5 * math.sin(2*x) + 0.25

In [6]:
@njit
def monte_carlo_numba_cpu(n_points, x_min=0, x_max=2*math.pi, y_min=0, y_max=2):
    count_inside = 0
    np.random.seed(42) 
    
    for _ in range(n_points):
        x = np.random.uniform(x_min, x_max)
        y = np.random.uniform(y_min, y_max)
        
        if y <= function_numba(x):
            count_inside += 1
    area_rectangle = (x_max - x_min) * (y_max - y_min)
    area_under_curve = area_rectangle * count_inside / n_points
    return area_under_curve

# CUDA Numba

In [7]:
@cuda.jit
def monte_carlo_cuda_kernel(results, n_points, x_min, x_max, y_min, y_max):
    thread_id = cuda.grid(1)
    threads_per_block = cuda.blockDim.x
    block_id = cuda.blockIdx.x
    idx = block_id * threads_per_block + thread_id # type: ignore
    
    if idx < n_points:
        seed = (idx + 1) * 12345
        x = x_min + (x_max - x_min) * ((seed * 1103515245 + 12345) & 0x7fffffff) / 0x7fffffff
        y = y_min + (y_max - y_min) * (((seed + 1) * 1103515245 + 12345) & 0x7fffffff) / 0x7fffffff
        
        fx = math.sin(x) + 0.5 * math.sin(2*x) + 0.25
        
        if y <= fx:
            results[idx] = 1
        else:
            results[idx] = 0

In [8]:
def monte_carlo_numba_cuda(n_points, x_min=0, x_max=2*math.pi, y_min=0, y_max=2):
    results = cuda.device_array(n_points, dtype=np.int32) # type: ignore
    
    threads_per_block = 256
    blocks_per_grid = (n_points + threads_per_block - 1) // threads_per_block
    
    monte_carlo_cuda_kernel[blocks_per_grid, threads_per_block]( # type: ignore
        results, n_points, x_min, x_max, y_min, y_max
    )
    
    results_host = results.copy_to_host() # опять на cpu
    
    count_inside = np.sum(results_host)
    area_rectangle = (x_max - x_min) * (y_max - y_min)
    area_under_curve = area_rectangle * count_inside / n_points
    
    return area_under_curve

# Numpy

In [9]:
def monte_carlo_numpy(n_points, x_min=0, x_max=2*math.pi, y_min=0, y_max=2):
    np.random.seed(42)
    
    x = np.random.uniform(x_min, x_max, n_points)
    y = np.random.uniform(y_min, y_max, n_points)
    
    fx = np.sin(x) + 0.5 * np.sin(2*x) + 0.25
    count_inside = np.sum(y <= fx)
    
    area_rectangle = (x_max - x_min) * (y_max - y_min)
    area_under_curve = area_rectangle * count_inside / n_points
    return area_under_curve

# Сравнение!

In [10]:
def measure_time(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    return result, end_time - start_time

In [12]:
exact_area = 2.356194490192345 

methods = [
    ("Pure Python", monte_carlo_python),
    ("NumPy", monte_carlo_numpy),
    ("Numba CPU", monte_carlo_numba_cpu),
    ("Numba CUDA", monte_carlo_numba_cuda),
]

results = {}

for name, method in methods:
    try:
        if name == "Numba CUDA" and not cuda.is_available():
            print(f"{name}: CUDA не доступна")
            continue
            
        method(100) 
        
        result, execution_time = measure_time(method, 1000000)
        error = abs(result - exact_area)
        
        results[name] = {
            'result': result,
            'time': execution_time,
            'error': error
        }
        
        print(f"{name:15} | Результат: {result:.8f} | "
                f"Время: {execution_time:.6f} сек | "
                f"Ошибка: {error:.8f}")
                
    except Exception as e:
        print(f"{name}: Ошибка - {str(e)}")

Pure Python     | Результат: 2.95393904 | Время: 4.801298 сек | Ошибка: 0.59774455
NumPy           | Результат: 2.95356205 | Время: 0.072565 сек | Ошибка: 0.59736756
Numba CPU       | Результат: 2.95736966 | Время: 0.052842 сек | Ошибка: 0.60117517
Numba CUDA: Ошибка - ERROR_INTERNAL (6)
Linker error log: ERROR 4 in nvvmAddNVVMContainerToProgram, may need newer version of nvJitLink library
 
